In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/sample_submission.csv
../data/test.csv
../data/train.csv


In [44]:
import torch
from torch import nn
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision.transforms import ToTensor, Lambda, Normalize, Compose
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchinfo import summary

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from tqdm import trange

In [4]:
class CustomDataset(Dataset):
    def __init__(self, input_file, device=torch.device('cpu'), image_size=(28,28), num_labels=10, train=True):
        self.df = pd.read_csv(input_file)
        self.image_size = image_size
        self.train = train
        self.device = device
        # print(input_file, image_size, num_labels, train)
        self.transform = Compose([
            ToTensor(),
            Normalize((0.5,), (0.5,))
        ])
        self.target_transform_bkp = Lambda(
            lambda y: torch.zeros(num_labels, dtype=torch.float).scatter_(0,torch.tensor(y), value=1)
        )
        self.target_transform = Lambda(
            lambda y: torch.tensor(y, dtype=torch.long)
        )
        

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        input_image, input_label = None, None
        if(self.train):
            input_image = self.df.iloc[idx, 1:].to_numpy().reshape(self.image_size)
            input_label = self.df.iloc[idx, 0]
        else:
            input_image = self.df.iloc[idx, :].to_numpy().reshape(self.image_size)

        input_image = input_image.astype(np.float32)
        input_image = self.transform(input_image).to(torch.float).to(device)
        if(input_label is not None):input_label = self.target_transform(input_label).to(device)

        # print(input_image.dtype)

        return {
            "image": input_image, "label":input_label
        }


In [5]:
def evaluate(model, dataloader, loss_fn, batch_size, evaluation_prefix=''):
    model.eval()
    
    loss, correct = 0, 0
    dataset_size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for input_batch in tqdm(dataloader, dynamic_ncols=True, desc=f"{evaluation_prefix} Evaluate", leave=False): #
            images, labels = input_batch['image'], input_batch['label']
    
            logits = model(images)
            # probabilities = nn.functional.softmax(logits, dim=1)
            
            loss += loss_fn(logits, labels)
            correct += ( logits.argmax(1) == labels ).type(torch.float).sum().item()

        loss /= num_batches
        correct /= dataset_size

        print(f"{evaluation_prefix} Evaluation : loss={loss.item():>8f} Accuracy={correct*100:>.1f} %")

    model.train()

    return correct*100, loss

In [6]:
!mkdir ./modelWeights/ -p

In [126]:
def train(
    model,
    train_dataloader, val_dataloader,
    epochs=1,
    learning_rate=1e-3,
    epoch_offset=0,
    save_checkpoint=True
):
    dataset_size = len(train_dataloader.dataset)
    loss = 0 # binary cross entropy
    loss_fn = nn.CrossEntropyLoss()
    optimizer = None # Adam, RMSProp
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3)
    
    model.train()
    for eidx in range(1,epochs+1):
        # print(f"Epoch: {eidx}")
        pbar = tqdm(train_dataloader, desc=f"Epoch: {eidx+epoch_offset}", dynamic_ncols=True)
        for batch, input_batch in enumerate(pbar):
            images, labels = input_batch['image'], input_batch['label']
    
            logits = model(images)
            # probabilities = nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if(batch % 10 == 0):
                loss_value = loss.item()
                pbar.set_postfix_str(f"loss: {loss_value:>7f}")
                # current = batch*batch_size+len(images)
                # print(f"epoch : {eidx}:{current*100/dataset_size:02.2f}, train loss : {loss_value}, ")
                # print(f"loss: {loss_value:>7f}  [{current:>5d}/{dataset_size:>5d}]")
                


        train_acc, train_loss = evaluate(model, train_dataloader, loss_fn, batch_size, evaluation_prefix="Training data")
        val_acc, val_loss = evaluate(model, val_dataloader, loss_fn, batch_size, evaluation_prefix="Validation data")

        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        new_lr = optimizer.param_groups[0]['lr']
        
        print(f"lr : {old_lr} -> {new_lr}")

        if(save_checkpoint):
            model_output_path = f"../modelWeights/SimpleCNN_epoch{eidx+epoch_offset:02d}_Acc{val_acc:>.1f}.pth"
            # model_output_path = f"./modelWeights/SimpleANN_epoch{eidx+epoch_offset:02d}.pth"
            print(f"saving : {model_output_path}")
            torch.save(model.state_dict(), model_output_path)

In [127]:
def predict(model, device, dataset):
    model.eval()
    outputs = pd.DataFrame(columns=["ImageId","Label"])
    with torch.no_grad():
        for idx in tqdm(range(len(dataset)), dynamic_ncols=True, desc="Predict"):
            image = dataset[idx]['image'].unsqueeze(0)
            logits = model(image)
            prob = torch.softmax(logits, dim=1)
            pred_class = prob.argmax(1).cpu().item()
            outputs.loc[len(outputs)] = [(idx+1), pred_class]

    return outputs


In [128]:
def showData(dataset):
    cols, rows = 5, 5

    fig = plt.figure(figsize=(10,10))
    
    for idx in range(1,cols*rows+1):
        sample_idx = torch.randint(len(dataset), size=(1,)).item()
        sample = dataset[sample_idx]
        image, label = sample['image'], sample['label']
        sub_title = label.item() if(label is not None) else idx 
        fig.add_subplot(rows, cols, idx)
        plt.title(sub_title)
        plt.axis('off')
        plt.imshow(image.squeeze().cpu(), cmap='gray')


In [129]:
class SimpleANN(nn.Module):
    def __init__(self, input_image_size=(28,28)):
        super().__init__()
        self.flatten = nn.Flatten()
        self.LinearRelu_1 = nn.Sequential(
            nn.Linear(input_image_size[0]*input_image_size[1], 512),
            nn.Dropout(0.2),
            nn.ReLU()
        )
        self.LinearRelu_2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.Dropout(0.2),
            nn.ReLU()
        )
        self.Linear_out = nn.Linear(512, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.LinearRelu_1(x)
        x = self.LinearRelu_2(x)
        logits = self.Linear_out(x)
        
        return logits


In [52]:
class ConvBnReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=0):
        super().__init__()
        self.convbnrelu = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, (kernel_size,)*2, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.convbnrelu(x)
        return x
        

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.convbnrelu_1 = ConvBnReLU(1,8,3,1)
        self.down1 = nn.MaxPool2d(2)
        self.convbnrelu_2 = ConvBnReLU(8,64,3,1)
        self.down2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.linearRelu_1 = nn.Sequential(
            nn.Linear(3136, 512),
            # nn.Dropout(0.2),
            nn.ReLU()
        )
        self.linear_out = nn.Linear(512, 10)

    def forward(self, x):
        x = self.convbnrelu_1(x) # [n,1,28,28] --> [n,8,28,28]
        x = self.down1(x) # [n,8,28,28] --> [n,8,14,14]
        x = self.convbnrelu_2(x) # [n,8,14,14] --> [n,64,14,14]
        x = self.down2(x) # [n,64,14,14] --> [n,64,7,7]
        x = self.flatten(x) # [n,64,7,7] --> [n, 3136]
        x = self.linearRelu_1(x)
        x = self.linear_out(x)
        
        return x

In [53]:
# 64*7*7

In [54]:
cnn_model = SimpleCNN()

summary(cnn_model, input_size=(1,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [1, 10]                   --
├─ConvBnReLU: 1-1                        [1, 8, 28, 28]            --
│    └─Sequential: 2-1                   [1, 8, 28, 28]            --
│    │    └─Conv2d: 3-1                  [1, 8, 28, 28]            80
│    │    └─BatchNorm2d: 3-2             [1, 8, 28, 28]            16
│    │    └─ReLU: 3-3                    [1, 8, 28, 28]            --
├─MaxPool2d: 1-2                         [1, 8, 14, 14]            --
├─ConvBnReLU: 1-3                        [1, 64, 14, 14]           --
│    └─Sequential: 2-2                   [1, 64, 14, 14]           --
│    │    └─Conv2d: 3-4                  [1, 64, 14, 14]           4,672
│    │    └─BatchNorm2d: 3-5             [1, 64, 14, 14]           128
│    │    └─ReLU: 3-6                    [1, 64, 14, 14]           --
├─MaxPool2d: 1-4                         [1, 64, 7, 7]             --
├─Flatten: 

In [56]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device


device(type='cuda')

In [57]:
# !ls /kaggle/input/digitrecognizer_simpleann_1/pytorch/default/1/

In [58]:
batch_size=64
loss_fn = nn.CrossEntropyLoss()

In [59]:
dataset = CustomDataset(input_file='../data/train.csv', device=device, train=True)

In [60]:
val_perc = 0.2
train_perc = 1 - val_perc

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_perc, val_perc], generator)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [61]:
input_batch = dataset[0]

In [122]:
# model = SimpleANN()
model = SimpleCNN()
model.to(device)
evaluate(model, val_dataloader, loss_fn, batch_size, "Validation data");

Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=22.835941 Accuracy=13.5 %


In [123]:
!rm -rf ../modelWeights/*.pth

In [124]:
final_dataset = CustomDataset("../data/train.csv", device=device, train=True)
final_train_dataset, final_val_dataset = random_split(dataset, [0.8, 0.2])

final_train_dataloader = DataLoader(final_train_dataset, batch_size=64, shuffle=True)
final_val_dataloader = DataLoader(final_val_dataset, batch_size=64, shuffle=True)

In [125]:
len(final_dataset), len(final_train_dataset), len(final_val_dataset)

(42000, 33600, 8400)

In [130]:
train(
    model,
    final_train_dataloader, final_val_dataloader,
    epochs=100, learning_rate=1e-3, epoch_offset=0,
    save_checkpoint=True
)

Epoch: 1:   0%|                                                  | 0/525 [00:00<?, ?it/s]

Training data Evaluate:   0%|                                    | 0/525 [00:00<?, ?it/s]

Training data Evaluation : loss=0.679627 Accuracy=88.3 %


Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=0.677427 Accuracy=88.2 %
lr : 0.001 -> 0.001
saving : ../modelWeights/SimpleCNN_epoch01_Acc88.2.pth


...

Epoch: 100:   0%|                                                | 0/525 [00:00<?, ?it/s]

Training data Evaluate:   0%|                                    | 0/525 [00:00<?, ?it/s]

Training data Evaluation : loss=0.031417 Accuracy=99.2 %


Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=0.049750 Accuracy=98.5 %
lr : 0.0005 -> 0.0005
saving : ../modelWeights/SimpleCNN_epoch100_Acc98.5.pth


In [131]:
test_dataset = CustomDataset("../data/test.csv", train=False)

In [132]:
outputs = predict(model, device, test_dataset)

Predict:   0%|                                                 | 0/28000 [00:00<?, ?it/s]

In [133]:
outputs.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [134]:
outputs.to_csv("../submissions//output_SimpleCNN_epoch100_acc98.5.csv",index=False)

!head -n6 ../submissions//output_SimpleCNN_epoch100_acc98.5.csv

ImageId,Label
1,2
2,0
3,9
4,9
5,3


In [135]:
!kaggle competitions submit -c digit-recognizer -f ../submissions//output_SimpleCNN_epoch100_acc98.5.csv -m "SimpleCNN_epoch103"

100%|████████████████████████████████████████| 208k/208k [00:03<00:00, 66.5kB/s]
400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/digit-recognizer


In [106]:
model = SimpleCNN()
model.load_state_dict(torch.load("./modelWeights/SimpleANN_epoch75_Acc98.2.pth"))
model.to(device)

outputs_2 = predict(model, device, test_dataset)
print(outputs_2.head())

outputs_2.to_csv("../submissions//output_SimpleCNN_epoch75_acc98.2.csv",index=False)
!head -n6 "../submissions//output_SimpleCNN_epoch75_acc98.2.csv"

Predict:   0%|                                                 | 0/28000 [00:00<?, ?it/s]

   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3
ImageId,Label
1,2
2,0
3,9
4,9
5,3
